Create and populate subset of source table

In [1]:
-- Step 1: Create a new table for the subset
CREATE TABLE walmart_ecommerce_subset (
    id INT IDENTITY(1,1) PRIMARY KEY,
    concatenated_text NVARCHAR(MAX),
    embedding VECTOR(1536) -- Adjusted for Ada embeddings
);

-- Step 2: Populate the new table with top N distinct concatenated values
INSERT INTO walmart_ecommerce_subset (concatenated_text)
SELECT DISTINCT TOP (100) 
    CONCAT(product_name, ' ', description) AS concatenated_text
FROM walmart_ecommerce_product_details
WHERE product_name IS NOT NULL AND description IS NOT NULL;


(100 rows affected)

Total execution time: 00:00:00.330

Create embeddings proc

In [4]:
CREATE OR ALTER PROCEDURE [dbo].[GetEmbeddings]
    @InputText NVARCHAR(MAX),
    @OutputText NVARCHAR(MAX) OUTPUT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @Payload NVARCHAR(MAX);
    SET @Payload = JSON_OBJECT('input': @InputText);

    BEGIN TRY
        EXEC sp_invoke_external_rest_endpoint
            @url = 'https://openaiappliedai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
            @method = 'POST',
            @credential = 'https://openaiappliedai.openai.azure.com/',
            @payload = @Payload,
            @response = @OutputText OUTPUT;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH
END;


Commands completed successfully.

Total execution time: 00:00:00.035

Batch process embeddings

In [ ]:
DECLARE @ID INT;
DECLARE @InputText NVARCHAR(MAX);
DECLARE @OutputText NVARCHAR(MAX);

-- Temp table for processing queue
IF OBJECT_ID('tempdb..#ProcessingQueue') IS NOT NULL DROP TABLE #ProcessingQueue;
CREATE TABLE #ProcessingQueue (
    ID INT PRIMARY KEY,
    InputText NVARCHAR(MAX)
);

-- Populate the queue with rows that need embeddings
INSERT INTO #ProcessingQueue (ID, InputText)
SELECT ID, concatenated_text
FROM walmart_ecommerce_subset
WHERE embedding IS NULL;

-- Process rows
WHILE EXISTS (SELECT 1 FROM #ProcessingQueue)
BEGIN
    -- Fetch the next row from the queue
    SELECT TOP 1 @ID = ID, @InputText = InputText
    FROM #ProcessingQueue;

    -- Call the embedding procedure
    --PRINT @InputText;
    EXEC dbo.GetEmbeddings @InputText, @OutputText OUTPUT;
    --PRINT @InputText + ': ' + CAST(CAST(JSON_QUERY(@OutputText, '$.result.data[0].embedding') AS VECTOR(1536)) as NVARCHAR(MAX));

    -- Update the embedding column in the subset table
    UPDATE walmart_ecommerce_subset
    SET embedding = CAST(JSON_QUERY(@OutputText, '$.result.data[0].embedding') AS VECTOR(1536))
    WHERE ID = @ID;

    -- Remove the processed row from the queue
    DELETE FROM #ProcessingQueue WHERE ID = @ID;

    -- Optional: Add a small delay for rate limiting
    WAITFOR DELAY '00:00:01'; -- Adjust as needed
END;


Generate Embeddings for Search Text

In [11]:
-- Search Text
DECLARE @text NVARCHAR(MAX) = 'Ebe Men Black Rectangle Half Rim Spring Hinge Eyewear Reading Glasses 2036: ...';
DECLARE @retval INT, @response NVARCHAR(MAX);
DECLARE @payload NVARCHAR(MAX);
SET @payload = JSON_OBJECT('input': @text);

-- Call OpenAI API to Get the Embedding
BEGIN TRY
    EXEC @retval = sp_invoke_external_rest_endpoint
        @url = 'https://openaiappliedai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
        @method = 'POST',
        @credential = 'https://openaiappliedai.openai.azure.com/',
        @payload = @payload,
        @response = @response OUTPUT;
END TRY
BEGIN CATCH
    SELECT 
        'SQL' AS error_source, 
        ERROR_NUMBER() AS error_code,
        ERROR_MESSAGE() AS error_message;
    RETURN;
END CATCH

-- Handle Errors
IF (@retval != 0) BEGIN
    SELECT 
        'OPENAI' AS error_source, 
        JSON_VALUE(@response, '$.result.error.code') AS error_code,
        JSON_VALUE(@response, '$.result.error.message') AS error_message,
        @response AS error_response;
    RETURN;
END;

-- Store Response in Table (Optional)
DROP TABLE IF EXISTS dbo.http_response;
CREATE TABLE dbo.http_response (response NVARCHAR(MAX));
INSERT INTO dbo.http_response (response) VALUES (@response);

-- Extract Embedding from Response
DECLARE @SearchEmbedding VECTOR(1536);
SET @SearchEmbedding = CAST(JSON_QUERY(@response, '$.result.data[0].embedding') AS VECTOR(1536));


(1 row affected)

Total execution time: 00:00:00.166

Perform Similiarity Search

In [13]:
-- Set parameters
DECLARE @top INT = 50; -- Limit the number of results
DECLARE @min_similarity DECIMAL(19, 16) = 0.75; -- Minimum similarity threshold

-- Fetch query vector from the latest embedding response
DECLARE @qv VECTOR(1536) = (
    SELECT TOP(1)
        CAST(JSON_QUERY(response, '$.result.data[0].embedding') AS VECTOR(1536)) AS query_vector
    FROM dbo.http_response
);

-- Perform similarity search on the subset table
SELECT TOP (@top)
    s.id,
    s.concatenated_text,
    vector_distance('cosine', @qv, s.embedding) AS distance
FROM dbo.walmart_ecommerce_subset s
WHERE vector_distance('cosine', @qv, s.embedding) <= 1 - @min_similarity
ORDER BY distance ASC;


(12 rows affected)

Total execution time: 00:00:00.046

id,concatenated_text,distance
1,"Ebe Men Black Rectangle Half Rim Spring Hinge Eyewear Reading Glasses 2036 Count on EBE for all of your eye correction needs this light weight sleek looking pair of glasses are something to be won. We guarantee you will love the style of these frames with the sharply colored temples on the side of your face give a great fashion look and a great comfort your face we guarantee your satisfaction.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all of our frames and lenses in our FDA listed Labs. Lenses Include Anti-Glare and Anti-Reflective Coating. You will Love your New Glasses and we Guarantee it 100%. We do not sell Cheap &quot;drug store&quot; reading glasses. Lens Width=55, Lens Height=29, Bridge Width=18, Temple Length=140, Frame Width=138",0.07657944997478905
4,"Ebe Men Silver Rectangle Half Rim Spring Hinge Reading Glasses a963 Count on EBE for all of your eye correction needs this light weight sleek looking pair of glasses are something to be won. We guarantee you will love the style of these frames with the sharply colored temples on the side of your face give a great fashion look and a great comfort your face we guarantee your satisfaction.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all of our frames and lenses in our FDA listed Labs. Lenses Include Anti-Glare and Anti-Reflective Coating. You will Love your New Glasses and we Guarantee it 100%. We do not sell Cheap &quot;drug store&quot; reading glasses. Lens Width=52, Lens Height=27, Bridge Width=16, Temple Length=138, Frame Width=130",0.09326433205059259
37,"Ebe Men Gold Shield Half Rim Spring Hinge Reading Glasses a970 Count on EBE for all of your eye correction needs this light weight sleek looking pair of glasses are something to be won. We guarantee you will love the style of these frames with the sharply colored temples on the side of your face give a great fashion look and a great comfort your face we guarantee your satisfaction.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all of our frames and lenses in our FDA listed Labs. Lenses Include Anti-Glare and Anti-Reflective Coating. You will Love your New Glasses and we Guarantee it 100%. We do not sell Cheap &quot;drug store&quot; reading glasses. Lens Width=50, Lens Height=27, Bridge Width=18, Temple Length=138, Frame Width=129",0.10784077920543012
97,"Ebe Prescription Glasses Mens Womens Burgundy Black Rectangular Half Rim Anti Glare grade s3130 Burgundy half-rim reading glasses. These unique frames feature a stylish half-frame design with soft rubber nose pieces, standard hinges for durability, and narrow black temple arms with built-in ear pieces. These classic looking frames are perfect for any situation, great for all day wear and comfort.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade 

Add to precedure

In [17]:
CREATE OR ALTER PROCEDURE dbo.SearchSimilarProductsSubset
    @SearchText NVARCHAR(MAX),
    @TopN INT = 50,
    @MinSimilarity DECIMAL(19, 16) = 0.75
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @response NVARCHAR(MAX);
    DECLARE @qv VECTOR(1536);
    DECLARE @retval INT;
    DECLARE @payload NVARCHAR(MAX);

    -- Prepare payload
    SET @payload = JSON_OBJECT('input': @SearchText);

    -- Call OpenAI API to get embedding
    BEGIN TRY
        EXEC @retval = sp_invoke_external_rest_endpoint
            @url = 'https://openaiappliedai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
            @method = 'POST',
            @credential = 'https://openaiappliedai.openai.azure.com/',
            @payload = @payload,
            @response = @response OUTPUT;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH

    -- Extract query vector from response
    SET @qv = CAST(JSON_QUERY(@response, '$.result.data[0].embedding') AS VECTOR(1536));

    -- Perform similarity search on the subset table
    SELECT TOP (@TopN)
        s.id,
        s.concatenated_text,
        vector_distance('cosine', @qv, s.embedding) AS distance
    FROM dbo.walmart_ecommerce_subset s
    WHERE vector_distance('cosine', @qv, s.embedding) <= 1 - @MinSimilarity
    ORDER BY distance ASC;
END;


Commands completed successfully.

Total execution time: 00:00:00.039

Use Procedure

In [18]:
EXEC dbo.SearchSimilarProductsSubset 
    @SearchText = 'Men''s gold glasses',
    @TopN = 10,
    @MinSimilarity = 0.80;


Commands completed successfully.

Total execution time: 00:00:00.132

id,concatenated_text,distance
37,"Ebe Men Gold Shield Half Rim Spring Hinge Reading Glasses a970 Count on EBE for all of your eye correction needs this light weight sleek looking pair of glasses are something to be won. We guarantee you will love the style of these frames with the sharply colored temples on the side of your face give a great fashion look and a great comfort your face we guarantee your satisfaction.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all of our frames and lenses in our FDA listed Labs. Lenses Include Anti-Glare and Anti-Reflective Coating. You will Love your New Glasses and we Guarantee it 100%. We do not sell Cheap &quot;drug store&quot; reading glasses. Lens Width=50, Lens Height=27, Bridge Width=18, Temple Length=138, Frame Width=129",0.18265632346990646
14,"Eye Buy Express Prescription Glasses Mens Womens Burgundy Crystal Clear Yellow Rounded Rectangular Reading Glasses Anti Glare grade Rounded rectangular cat-eye reading glasses. These frames feature a deep burgundy color on the face of the frame with crystal interior, as well as crystal temple detail and yellow, turqoise and black marbled colors on the inside of the temple arms. The standard hinges provide durability and the TR-90 con struction allows for some flex making these glasses very sturdy and perfect for every day use!| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all of our frames and lenses in our FDA listed Labs. Lenses Include Anti-Glare and Anti-Reflective Coating. You will Love your New Glasses and we Guarantee it 100%. We do not sell Cheap &quot;drug store&quot; reading glasses. Lens Width=54, Lens Height=37, Bridge Width=15, Temple Length=144, Frame Width=135",0.18657926411160963
72,"Ebe Reading Glasses Mens Womens Tortoise Bold Rectangular Full Frame Anti Glare grade ckbdp9088 Rectangular tortoise-shell colored readers with full frames. These spectacles feature a one-piece frame with built-in matching color bridge and nose pieces for added durability, leading into interesting gold colored stainless steel temple detail with standard hinge for durability, thin gold colored temple arms, and chocolatey brown soft rubber ear pieces with shiny silver tips for added detail and style. Perfect for anyone looking for a solid bold pair of reading glasses for all day use and wearaibility without &quot;hot spots&quot; or pinching.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design and produce all of our frames and lenses in our FDA listed Labs. Lenses Include Anti-Glare and Anti-Reflective Coating. You will Love your New Glasses and we Guarantee it 100%. We do not sell Cheap &quot;drug store&quot; reading glasses. Lens Width=52, Lens Height=32, Bridge Width=15, Temple Length=145, Frame Width=130",0.1925752838454139
61,"Ebe Reading Glasses Mens Womens Gold Harry Potter Style Horned Rim Anti Glare Cozy zsm5500 We hope you love these spectac

BAsic call to chat endpoint

In [30]:
-- Step 1: Define the user query
DECLARE @text NVARCHAR(MAX) = 'What are the best products for organizing a birthday party for a teenager girl?';

-- Step 2: Create the payload for the Chat endpoint
DECLARE @payload NVARCHAR(MAX);
SET @payload = JSON_OBJECT(
    'messages': JSON_ARRAY(
        JSON_OBJECT(
            'role': 'system',
            'content': 'You are a system assistant that helps users answer questions using product data provided to you.'
        ),
        JSON_OBJECT(
            'role': 'user',
            'content': @text
        )
    ),
    'max_tokens': 800,
    'temperature': 0.7,
    'frequency_penalty': 0,
    'presence_penalty': 0,
    'top_p': 0.95,
    'stop': NULL
);

-- Step 3: Call the Chat endpoint
DECLARE @retval INT, @response NVARCHAR(MAX);
EXEC @retval = sp_invoke_external_rest_endpoint
    @url = 'https://openaiappliedai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview',
    @headers = '{"Content-Type":"application/json"}',
    @method = 'POST',
    @credential = 'https://openaiappliedai.openai.azure.com/',
    @timeout = 120,
    @payload = @payload,
    @response = @response OUTPUT;

-- Call the Chat endpoint (simplified for clarity)
--DECLARE @response NVARCHAR(MAX);
-- Assume @response is populated by the API call as before

-- Step 1: Parse the `choices` array
DROP TABLE IF EXISTS #ChatResponse;
SELECT * 
INTO #ChatResponse
FROM OPENJSON(@response, '$.result.choices');

-- Step 2: Extract the `message.content` field
SELECT t.[value] AS AssistantResponse
FROM #ChatResponse c
CROSS APPLY OPENJSON(c.value, '$.message') AS t
WHERE t.[key] = 'content';

-- Clean up
DROP TABLE IF EXISTS #ChatResponse;






(1 row affected)

(1 row affected)

Total execution time: 00:00:05.733

AssistantResponse
"When organizing a birthday party for a teenage girl, it's important to choose products that are trendy, fun, and suitable for the theme of the party. Here are some recommended products that can help make the event memorable: 1. **Themed Decorations**: - **Banners and Streamers**: Look for customizable banners that can include the birthday girl's name and age. - **Balloon Sets**: Choose balloon bouquets that match the party theme, whether it's pastel colors, metallics, or a specific character theme. 2. **Tableware**: - **Disposable Plates and Cups**: Opt for stylish designs that fit the theme, such as floral patterns, glitter, or favorite characters. - **Napkins and Tablecloths**: Coordinate with the overall color scheme for a cohesive look. 3. **Party Favors**: - **Personalized Goodie Bags**: Fill them with fun items like lip gloss, friendship bracelets, or mini notepads. - **DIY Kits**: Consider craft kits that guests can take home and create. 4. **Lighting**: - **String Lights**: These can add a magical touch to the party space, especially if it's outdoors or in a dimly lit area. - **LED Party Lights**: For a dance party atmosphere, disco lights can enhance the vibe. 5. **Games and Activities**: - **Photo Booth Props**: Include fun hats, glasses, and signs for memorable photo opportunities. - **Party Games**: Card games or board games that are popular among teens can keep everyone entertained. 6. **Cake and Treats**: - **Custom Cake or Cupcake Toppers**: Personalize the cake with a theme-related topper. - **Dessert Table Setup**: Use tiered stands and decorative jars for candy and treats. 7. **Music and Entertainment**: - **Bluetooth Speaker**: A portable speaker for playing music during the party. - **Karaoke Machine**: If the birthday girl enjoys singing, this could be a hit. 8. **Invitations**: - **Digital Invitations**: Consider using online platforms to create and send stylish invitations that can be easily shared. 9. **Seating Arrangements**: - **Themed Cushions or Blankets**: Add comfort and style to seating areas with thematic décor. 10. **Photo and Video**: - **Instant Camera**: For capturing memories on the spot with friends. - **Polaroid or Disposable Cameras**: These can be fun for guests to use throughout the event. By selecting products that cater to the birthday girl's interests and preferences, you can create an engaging and enjoyable party atmosphere. Don't forget to consider the guest list and the available space when planning your setup!"


Create PROC for completions

In [33]:
CREATE OR ALTER PROCEDURE dbo.GetCompletion
    @InputText NVARCHAR(MAX), -- Single input text containing the query and any additional context
    @CompletionResponse NVARCHAR(MAX) OUTPUT
AS
BEGIN
    SET NOCOUNT ON;

    -- Variables
    DECLARE @payload NVARCHAR(MAX);
    DECLARE @response NVARCHAR(MAX);
    DECLARE @retval INT;

    -- Step 1: Prepare Chat API Payload
    SET @payload = JSON_OBJECT(
        'messages': JSON_ARRAY(
            JSON_OBJECT(
                'role': 'system',
                'content': '
                    You are a system assistant helping users find the best products based on their query. 
                    Use the provided context to craft a helpful response.
                '
            ),
            JSON_OBJECT(
                'role': 'user',
                'content': @InputText
            )
        ),
        'max_tokens': 800,
        'temperature': 0.7,
        'frequency_penalty': 0,
        'presence_penalty': 0,
        'top_p': 0.95,
        'stop': NULL
    );

    -- Step 2: Call Chat Endpoint
    BEGIN TRY
        EXEC @retval = sp_invoke_external_rest_endpoint
            @url = 'https://openaiappliedai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview',
            @headers = '{"Content-Type":"application/json"}',
            @method = 'POST',
            @credential = 'https://openaiappliedai.openai.azure.com/',
            @timeout = 120,
            @payload = @payload,
            @response = @response OUTPUT;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH

    -- Step 3: Parse Assistant's Response
    DROP TABLE IF EXISTS #ChatResponse;
    SELECT * INTO #ChatResponse
    FROM OPENJSON(@response, '$.result.choices');

    SELECT @CompletionResponse = t.[value]
    FROM #ChatResponse c
    CROSS APPLY OPENJSON(c.value, '$.message') AS t
    WHERE t.[key] = 'content';

    -- Clean up
    DROP TABLE IF EXISTS #ChatResponse;
END;


Commands completed successfully.

Total execution time: 00:00:00.040

CREATE FINAL Proc to handle end to end recommendation

In [39]:
CREATE OR ALTER PROCEDURE dbo.RAGSearchWithChat
    @SearchText NVARCHAR(MAX), -- User's search query
    @TopN INT = 10,           -- Number of top results to retrieve
    @MinSimilarity DECIMAL(19, 16) = 0.75 -- Minimum similarity threshold
AS
BEGIN
    SET NOCOUNT ON;

    -- Variables
    DECLARE @qv VECTOR(1536);
    DECLARE @FormattedResults NVARCHAR(MAX);
    DECLARE @InputText NVARCHAR(MAX);
    DECLARE @AssistantResponse NVARCHAR(MAX);
    DECLARE @response NVARCHAR(MAX);

    -- Step 1: Generate Query Vector using existing GetEmbeddings procedure
    EXEC dbo.GetEmbeddings 
        @InputText = @SearchText,
        @OutputText = @response OUTPUT;

    -- Extract the embedding
    SET @qv = CAST(JSON_QUERY(@response, '$.result.data[0].embedding') AS VECTOR(1536));

    -- Step 2: Perform Similarity Search
    SELECT TOP (@TopN)
        p.id,
        p.concatenated_text,
        vector_distance('cosine', @qv, p.embedding) AS similarity
    INTO #SearchResults
    FROM dbo.walmart_ecommerce_subset p
    WHERE vector_distance('cosine', @qv, p.embedding) <= 1 - @MinSimilarity
    ORDER BY similarity ASC;

    -- Check if there are any results
    IF NOT EXISTS (SELECT 1 FROM #SearchResults)
    BEGIN
        -- Provide a default answer if no matches are found
        SET @AssistantResponse = 'Sorry, I couldn''t find any matching products in the database. ' + 
                                 'Please try refining your search or providing more details.';
        SELECT @AssistantResponse AS AssistantResponse;

        -- Clean up and exit
        DROP TABLE IF EXISTS #SearchResults;
        RETURN;
    END;

    -- Step 3: Format the search results for the Chat prompt
    SELECT @FormattedResults = STRING_AGG(
        CAST(id AS NVARCHAR(10)) + '=>' + concatenated_text, CHAR(13) + CHAR(10)
    ) 
    FROM #SearchResults;

    -- Step 4: Prepare the Input Text for Chat Endpoint
    SET @InputText = 
        '## Products ##' + CHAR(13) + CHAR(10) +
        @FormattedResults + CHAR(13) + CHAR(10) +
        '## Query ##' + CHAR(13) + CHAR(10) +
        @SearchText;

    -- Step 5: Get Completion using GetCompletion procedure
    EXEC dbo.GetCompletion
        @InputText = @InputText,
        @CompletionResponse = @AssistantResponse OUTPUT;

    -- Step 6: Return Final Response
    SELECT @AssistantResponse AS AssistantResponse;

    -- Clean up
    DROP TABLE IF EXISTS #SearchResults;
END;


Commands completed successfully.

Total execution time: 00:00:00.044

TEST END to END

In [43]:
EXEC dbo.RAGSearchWithChat 
    @SearchText = 'what kind of shoes should women buy?',
    @TopN = 5,
    @MinSimilarity = 0.8;


Commands completed successfully.

Total execution time: 00:00:03.078

AssistantResponse
"When choosing shoes, women should consider their needs and preferences, such as comfort, style, and purpose. Here are two excellent options based on the provided context: 1. **Dr. Comfort Paradise Women's Casual Shoe**: This shoe features hand-tooled and burnished leather, offering a luxurious look suitable for both dressy and casual occasions. It also comes in a wide width (4.5 X-Wide), making it a great choice for those who need extra room. The Velcro closure provides ease of wear. 2. **Propet Breeze Walker Sandals**: If comfort and adjustability are priorities, the Propet Breeze Walker is a fantastic option. It has supple leather uppers with three adjustable straps for a customized fit. The contoured footbed and built-in arch support enhance comfort, making it ideal for walking or casual outings. Both options provide comfort and style, but the best choice will depend on whether you prefer a casual shoe or a sandal."
